# Importing Libraries

In [73]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer 

#nltk.download('all')

# Reading Docs

In [74]:
path = "Docs/"

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        files.append(os.path.join(r, file))
    
for f in files:
    print(f)

Docs/file1
Docs/file2
Docs/file3.txt


# Removing Headers and HTML Tags, Tokenization, Lower Casing, StopListing, Stemming

In [77]:
docs_words = {}
i = 0 #for docs_words

for f in files:
    doc = open(f,"r")
    html = doc.read()
    
    index = html.find("<html")
    if index == -1 :
        html.find("<Html")
    elif index == -1 : 
        html.find("<HTML")
    html = html[index:]

    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    
    #print("Doc : " + text + "\n\n")
    tknzr = TweetTokenizer()
    tokens = tknzr.tokenize(text)
    words_pos = {}
    j = 0; #for words_pos
    for t in tokens:
        t = t.lower() #lowering Case
        
        if t not in words_pos.keys():
            words_pos[t] = []
        if t in words_pos.keys():
            words_pos[t].append(j)
        j = j + 1
    
    docs_words[i] = words_pos
    i = i + 1

print(docs_words) 

{0: {'dark': [0, 33, 48, 98, 120, 167, 183, 228], "chocolate's": [1, 99, 328, 343, 372], 'health': [2, 7, 41, 100, 329, 346, 399], 'benefits': [3, 330], 'chocolate': [4, 19, 30, 34, 49, 66, 121, 168, 184, 212, 229, 270, 280, 287, 301, 309, 366, 375, 393, 397], 'and': [5, 50, 61, 83, 122, 140, 150, 156, 169, 198, 213, 288, 336, 367, 386, 394, 398], 'your': [6, 175, 214, 246, 259, 289, 312, 368, 395, 422], ':': [8], 'the': [9, 11, 54, 80, 93, 105, 161, 209, 255, 296, 349, 356, 391, 429, 432], 'basics': [10, 12, 350], "it's": [13], 'more': [14, 113, 253, 293, 322], 'than': [15, 294], 'wishful': [16], 'thinking': [17], '—': [18, 240], 'can': [20], 'be': [21, 172], 'good': [22, 173, 305], 'for': [23, 174, 219, 360], 'you': [24, 421, 435, 450], '.': [25, 42, 63, 90, 116, 142, 158, 177, 206, 216, 250, 262, 277, 286, 314, 340, 348, 378, 427, 448, 451], 'studies': [26, 94, 164, 347], 'show': [27], 'that': [28, 237, 439], 'eating': [29], ',': [31, 35, 47, 76, 111, 136, 138, 153, 182, 189, 193, 2

# Removing stop words

In [79]:
f = open("stoplist.txt", "r") 
stoplist = f.read().splitlines() #Stoplist words

for i in range(len(docs_words)):
    for key in list(docs_words[i]):
        if key in stoplist or len(key) == 1:
            docs_words[i].pop(key,None)
#print(docs_words)            

{0: {'dark': [0, 33, 48, 98, 120, 167, 183, 228], "chocolate's": [1, 99, 328, 343, 372], 'health': [2, 7, 41, 100, 329, 346, 399], 'benefits': [3, 330], 'chocolate': [4, 19, 30, 34, 49, 66, 121, 168, 184, 212, 229, 270, 280, 287, 301, 309, 366, 375, 393, 397], 'basics': [10, 12, 350], "it's": [13], 'wishful': [16], 'thinking': [17], 'good': [22, 173, 305], 'studies': [26, 94, 164, 347], 'show': [27], 'eating': [29], 'primarily': [32], 'contribute': [37], 'improved': [39, 190, 337], 'cardiovascular': [40, 151], 'packed': [43], 'natural': [45, 87, 131], 'antioxidants': [46, 115, 128, 149], 'cocoa': [51, 73, 109, 123, 170], 'sit': [52], 'good-for-you': [56], 'category': [57], 'green': [59], 'tea': [60], 'blueberries': [62], "that's": [64], 'comes': [67], 'cacao': [69, 81, 385], 'beans': [70, 74], 'grow': [78], 'tree': [82], 'full': [85], 'plant': [88], 'nutrients': [89, 285], 'date': [96], 'highlight': [97], 'values': [101], 'highest': [106], 'percentage': [107], 'solids': [110], 'flavano

# Stemming 